### Import the necessary libraries

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras import layers, models

### Loading the data

In [2]:
# load the data from the pickle file in the data/ directory
df = pd.read_pickle('data/words_df.pkl')

In [3]:
df.head()

,WordID,SegmentationResult,GrayLevel,BoundingBox,GrammaticalTag,Transcription,ImageData
0,a01-000u-00-00,ok,154,"(408, 768, 27, 51)",AT,A,"[[[0.9764706], [0.9764706], [0.9764706], [0.97..."
1,a01-000u-00-01,ok,154,"(507, 766, 213, 48)",NN,MOVE,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1..."
2,a01-000u-00-02,ok,154,"(796, 764, 70, 50)",TO,to,"[[[0.9843137], [0.9843137], [0.9843137], [0.98..."
3,a01-000u-00-03,ok,154,"(919, 757, 166, 78)",VB,stop,"[[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1..."
4,a01-000u-00-04,ok,154,"(1185, 754, 126, 61)",NPT,Mr.,"[[[0.9647059], [0.90588236], [0.80784315], [0...."


### Splitting train, validation, and test data

In [9]:
count=0
c2=0
for i, img in enumerate(df['ImageData']):
    try:
        if img.shape == (32, 128, 1):  # Replace with your expected dimensions
            count+=1
    except AttributeError:
        print(f"{df['WordID'][i]} will be deleted")
        df.drop([i], inplace=True)
        c2+=1
print(count)
print(c2)

a01-117-05-02 will be deleted
r06-022-03-05 will be deleted
115318
2


2 images data weren't loaded into the dataframe. Delete those rows.

In [26]:
print(len(df['Transcription'].unique()))

13550


In [27]:
# calculate the number of unique letters used in the values of the Transcription column
unique_letters = set()
for word in df['Transcription']:
    unique_letters.update(word)
print(len(unique_letters))

79


In [28]:
# calculate the longest word
max_len = 0
for word in df['Transcription']:
    if len(word) > max_len:
        max_len = len(word)
print(max_len)

53


In [15]:
# Split data into training and validation sets
X = np.stack(df['ImageData'].values)  # Convert the list of images to a numpy array
y = df['Transcription'].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [16]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Create a label encoder for the transcriptions
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# One-hot encode the labels
vocab_size = len(encoder.classes_)
y_encoded = np.eye(vocab_size)[y_encoded].astype(np.int32)

### Neural Networks

#### Starting simple

In [ ]:
# Define a simple neural network model
model = models.Sequential()
model.add(layers.Flatten(input_shape=(32, 128, 1)))  # Flatten the input
model.add(layers.Dense(128, activation='relu'))  # A dense layer with ReLU activation
model.add(layers.Dense(10, activation='softmax'))  # Output layer with softmax activation; change '10' to your number of classes

# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))